In [ ]:
import numpy as np
from mesh.mesh_read_plot3D import StructuredMeshInitialization2D
from mesh.mesh import MeshGeoCalculator2D
import boundary.boundary as bd
import type_transform as tf
import config
import Initialization as initial
from solver.residual import compute_residual_ausm
from solver.solver import RK3Solver

# 读取并预处理网格

In [ ]:
mesh_read = StructuredMeshInitialization2D()
mesh_read.load_file("airfoil0012extend.grd", "airfoil0012extend.inp", 0.001)
mesh_read.merge_blocks_2D()
mesh_read.interface_transform_cal()
mesh_read.mesh_reader.info()
mesh_read.bc_reader.info()
mesh_read.print_block_info()

# 计算网格几何参数

In [ ]:
mesh_geocal = MeshGeoCalculator2D(mesh_read)
mesh_geocal.compute_centroids()
mesh_geocal.compute_volumes()
mesh_geocal.compute_face_vectors()
for i, item in enumerate(mesh_geocal.mesh.blocks):
    print(f"block {i} keys: {list(item.keys())}")
print(mesh_geocal.mesh.blocks[0]['bc'])

# 添加虚网格

In [ ]:
blocks = np.copy(mesh_geocal.mesh.blocks)
bd.crate_ghost_cells(blocks, config.GHOST_LAYER, config.N_C)
for block in blocks:
 for bc in block['bc']: 
    if 'ghost_cell' in bc:
        print("ghost_cell shape:", bc['ghost_cell'].shape)
for i, block in enumerate(blocks):
    print(f"Block {i} keys:", list(block.keys()))

# 整理为一个用于计算的列表

In [ ]:
blocks_cal = tf.trans_list2numpy_2d(blocks, config.N_C)
for i, block in enumerate(blocks_cal):
    print(f"Block {i} keys:", list(block.keys()))
    print(f"Block {i} has {len(block['bc'])} boundary conditions.")
    for j, bc in enumerate(block['bc']):
        print(f"  BC {j} keys: {list(bc.keys())}")
        print(f"    → type: {bc.get('type', 'N/A')}")

# 初始化流场和边界条件

In [ ]:
initial.intialization_from_farfield(blocks_cal)
print(blocks_cal[0]['fluid'][0, 0, 0:4])
print(blocks_cal[0]['bc'][0]['ghost_cell'][0, 0, 0:4])

# 迭代计算

In [ ]:
slover = RK3Solver(blocks_cal, config.GAMMA, 0.5)
slover.run(5000, 1e-2)
blocks_result = slover.blocks

# 保存列表

In [ ]:
import pickle
with open('blocks_result.pkl', 'wb') as f:
    pickle.dump(blocks_result, f)

# 读取列表

In [ ]:
import pickle
with open('blocks_result.pkl', 'rb') as f:
    blocks_result = pickle.load(f)
for i, block in enumerate(blocks_result):
    print(f"Block {i} keys:", list(block.keys()))

# 继续计算

In [ ]:
slover = RK3Solver(blocks_result, config.GAMMA, 0.5)
slover.run(500, 1e-2)
blocks_result = slover.blocks

# 

# 输出为.dat文件

In [ ]:
from post_output.output_tecplot import output_tecplot
output_tecplot(blocks_result)